In [10]:
import pandas as pd
import math
from IPython.display import display

In [2]:
# importo i dati
def importCSV(path):
    return pd.read_csv(path, index_col = 0)
    
df = importCSV("testcsv.csv")
df

,Esercizio1,Esercizio2,Esercizio3
Studente,,,
Marco,8,7,5
Simone,7,5,4
Davide,6,5,9


In [3]:
# assegno True nelle caselle in cui il voto dell'esercizio è >= di 6 e False altrimenti

def replaceValueTF(df):
    dfAux = df.copy()
    flag = True
    for column in dfAux:
        dfAux.loc[dfAux[column] < 6, column] = 0
        dfAux.loc[dfAux[column] >= 6, column] = 1  
    return dfAux

In [7]:
# stimo l'abilità degli studenti con tecnica naif
# l'abilità è calcolata tramite media ponderata, i valori usati sono i punteggi degli studenti presi negli esercizi ed 
# i pesi sono le difficoltà assegnate agli esercizi

def getDiff(n):
    lsDiff = pd.DataFrame(columns = df.columns , index = ["Difficulty"])
    print("Insersci la difficoltà per i seguenti item: \n")
    for columns in lsDiff:
        lsDiff.loc["Difficulty",columns] = int(input(columns + ": "))
    return lsDiff

def estimateAbility(df,lsDiff):
    lsAb = pd.DataFrame(index = df.index , columns = ["Ability"])
    for index, row in df.iterrows():
        ability = (row*(lsDiff.loc["Difficulty"])).sum()/lsDiff.loc["Difficulty"].sum()
        lsAb.loc[index,"Ability"] = round(ability, 2)
    return lsAb       

def getDiscrim(n):
    lsDiscrim = pd.DataFrame(columns = df.columns , index = ["Discriminant"])
    print("Insersci i discriminanti per i seguenti item: \n")
    for columns in lsDiscrim:
        lsDiscrim.loc["Discriminant",columns] = int(input(columns + ": "))
    return lsDiscrim

def getGuess(n):
    lsGuess = pd.DataFrame(columns = df.columns , index = ["Guess"])
    print("Insersci le probabilità di indovinare i seguenti item: \n")
    for columns in lsGuess:
        lsGuess.loc["Guess",columns] = float(input(columns + ": "))
    return lsGuess    

In [5]:
# PL per calcolare la probabilità che ogni studente risponda correttamete ad ogni item

def pl1(lsAb,lsDiff):
    dfProb = pd.DataFrame(columns=df.columns, index=df.index)
    for index, row in dfProb.iterrows():
        for columns in dfProb:
            ability = lsAb.loc[index,"Ability"]
            difficulty = lsDiff.loc["Difficulty",columns]
            res = (math.exp(ability - difficulty)) / (1 + math.exp(ability - difficulty))
            dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
    return dfProb

def pl2(lsAb,lsDiff,lsDiscrim):
    dfProb = pd.DataFrame(columns=df.columns, index=df.index)
    for index, row in dfProb.iterrows():
        for columns in dfProb:
            ability = lsAb.loc[index,"Ability"]
            difficulty = lsDiff.loc["Difficulty",columns]      
            discriminant = lsDiscrim.loc["Discriminant",columns]
            res = (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
            dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
    return dfProb

def pl3(lsAb,lsDiff,lsDiscrim, lsGuess):
    dfProb = pd.DataFrame(columns=df.columns, index=df.index)
    for index, row in dfProb.iterrows():
        for columns in dfProb:
            ability = lsAb.loc[index,"Ability"]
            difficulty = lsDiff.loc["Difficulty",columns]      
            discriminant = lsDiscrim.loc["Discriminant",columns]            
            guessProb = lsGuess.loc["Guess",columns]
            res = guessProb + (1 - guessProb) * (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
            dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
    return dfProb

In [11]:
# scelta del pl e raccolta dei dati

while(True):
    pl = int(input("Scegliere il tipo di PL da utilizzare: "))
    print()
    if pl in range(1,4):
        numItems = len(df.columns)
        lsDiff = getDiff(numItems)
        lsAb = estimateAbility(df,lsDiff)
        if(pl == 1):
            dfProb = pl1(lsAb,lsDiff)
        else:
            print()
            lsDiscrim = getDiscrim(numItems)
            if(pl == 2):
                dfProb = pl2(lsAb,lsDiff,lsDiscrim)
            else:
                print()
                lsGuess = getGuess(numItems)
                dfProb = pl3(lsAb,lsDiff,lsDiscrim, lsGuess)
        break
    else:
        print("ERRORE: i tipi di PL sono solo 1,2 e 3")

if(pl == 1):
    display(lsDiff, lsAb, dfProb)
elif(pl == 2):
    display(lsDiff, lsAb, lsDiscrim, dfProb)
else:
    display(lsDiff, lsAb, lsGuess, dfProb)

Scegliere il tipo di PL da utilizzare: 4

ERRORE: i tipi di PL sono solo 1,2 e 3
Scegliere il tipo di PL da utilizzare: 3

Insersci la difficoltà per i seguenti item: 

Esercizio1: 5
Esercizio2: 6
Esercizio3: 7

Insersci i discriminanti per i seguenti item: 

Esercizio1: 1
Esercizio2: 1
Esercizio3: 1

Insersci le probabilità di indovinare i seguenti item: 

Esercizio1: 4
Esercizio2: 4
Esercizio3: 3


,Esercizio1,Esercizio2,Esercizio3
Difficulty,5,6,7


,Ability
Studente,
Marco,6.5
Simone,5.17
Davide,6.83


,Esercizio1,Esercizio2,Esercizio3
Guess,4,4,3


,Esercizio1,Esercizio2,Esercizio3
Studente,,,
Marco,154.73%,213.26%,224.49%
Simone,237.28%,308.91%,272.35%
Davide,141.47%,191.09%,208.48%
